<a href="https://colab.research.google.com/github/TWHessNM/DataSciencePython/blob/master/Copy_of_project_3_with_Google_Big_Query_Data_SQL_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adventures in Bitcoin

## Setup


In [1]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import plotly.express as px

auth.authenticate_user()

In [2]:
project_id = 'bitcoindata-381723'

# Create client object
client = bigquery.Client(project=project_id)

#Initial Exploration

### List the tables

In [3]:
# Construct a reference to the 'crypto_bitcoin' dataset
dataset_ref = client.dataset("crypto_bitcoin", project="bigquery-public-data")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [4]:
# Get all the tables in the dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables: 
  print(table.table_id)

blocks
inputs
outputs
transactions


### Table schema


In [5]:
# Construct a reference to the "transactions" table
table_ref = dataset.table("transactions")

# API request - fetch the table
table = client.get_table(table_ref)

# See the table's schema - name, field type, mode, description
table.schema

[SchemaField('hash', 'STRING', 'REQUIRED', None, 'The hash of this transaction', (), None),
 SchemaField('size', 'INTEGER', 'NULLABLE', None, 'The size of this transaction in bytes', (), None),
 SchemaField('virtual_size', 'INTEGER', 'NULLABLE', None, 'The virtual transaction size (differs from size for witness transactions)', (), None),
 SchemaField('version', 'INTEGER', 'NULLABLE', None, 'Protocol version specified in block which contained this transaction', (), None),
 SchemaField('lock_time', 'INTEGER', 'NULLABLE', None, 'Earliest time that miners can include the transaction in their hashing of the Merkle root to attach it in the latest block of the blockchain', (), None),
 SchemaField('block_hash', 'STRING', 'REQUIRED', None, 'Hash of the block which contains this transaction', (), None),
 SchemaField('block_number', 'INTEGER', 'REQUIRED', None, 'Number of the block which contains this transaction', (), None),
 SchemaField('block_timestamp', 'TIMESTAMP', 'REQUIRED', None, 'Timesta

In [6]:
dir(table.schema[0])

['_SchemaField__get_int',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fields',
 '_key',
 '_properties',
 'default_value_expression',
 'description',
 'field_type',
 'fields',
 'from_api_repr',
 'is_nullable',
 'max_length',
 'mode',
 'name',
 'policy_tags',
 'precision',
 'scale',
 'to_api_repr',
 'to_standard_sql']

In [7]:
# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,275,275,1,0,00000000dc55860c8a29c58d45209318fa9e9dc2c1833a...,181,2009-01-12 06:02:13+00:00,2009-01-01,1,2,4000000000.000000000,4000000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'f4184...","[{'index': 0, 'script_asm': '04b5abd412d4341b4..."
1,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,275,275,1,0,0000000054487811fc4ff7a95be738aa5ad9320c394c48...,182,2009-01-12 06:12:16+00:00,2009-01-01,1,2,3000000000.000000000,3000000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'a16f3...","[{'index': 0, 'script_asm': '0401518fa1d1e1e3e..."
2,12b5633bad1f9c167d523ad1aa1947b2732a865bf5414e...,276,276,1,0,00000000f46e513f038baf6f2d9a95b2a28d8a6c985bcf...,183,2009-01-12 06:34:22+00:00,2009-01-01,1,2,2900000000.000000000,2900000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': '591e9...","[{'index': 0, 'script_asm': '04baa9d3665315562..."
3,828ef3b079f9c23829c56fe86e85b4a69d9e06e5b54ea5...,276,276,1,0,00000000fb5b44edc7a1aa105075564a179d65506e2bd2...,248,2009-01-12 20:04:20+00:00,2009-01-01,1,2,2800000000.000000000,2800000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': '12b56...","[{'index': 0, 'script_asm': '04bed827d37474bef..."
4,35288d269cee1941eaebb2ea85e32b42cdb2b04284a56d...,277,277,1,0,00000000689051c09ff2cd091cc4c22c10b965eb8db3ad...,545,2009-01-15 05:48:32+00:00,2009-01-01,1,2,2500000000.000000000,2500000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'd71fd...","[{'index': 0, 'script_asm': '044a656f065871a35..."


##  Add safe config settings

In [8]:
ONE_MB = 1_000*1_000
ONE_GB = 1_000*ONE_MB

In [12]:
#Dry run query to estimate the size of a query before running it
query = """
        SELECT *
        FROM bigquery-public-data.crypto_bitcoin.transactions
        LIMIT 5
        """


dry_run_config = bigquery.QueryJobConfig(dry_run = True)
dry_run_query_job = client.query(query, job_config= dry_run_config)
size = dry_run_query_job.total_bytes_processed
print(f"{size:_}")

1_771_095_216_329


In [13]:
#Safe Config to limit scanned data.
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

safe_query_job = client.query(query, job_config=safe_config)
df = safe_query_job.to_dataframe()
df.head()

InternalServerError: ignored

In [14]:
df.shape

NameError: ignored

In [16]:
query = """
        SELECT *
        FROM bigquery-public-data.crypto_bitcoin.transactions
        LIMIT 5
        """
df = client.query(query).to_dataframe()
df.head()

Forbidden: ignored

## Come up with questions about your data
* What sort of information is in this dataset?
* How many records are there?
* Have the number of bitcoin transactions increased year over year?
* Does New Mexico get more or less rain now than 20 years ago?
* How many different countries (states, counties, cities, etc) have records in this data set?




## Use SQL queries to pull specific information

Do NOT pull all the data and then filter using DataFrame methods etc. Make sure and use AT LEAST 13 of the 15 SQL options listed below. (You may have to get creative and come up with more questions to ask/answer.)

* Basic Queries
  * SELECT (with * and with column names)
  * WHERE
  * AND
  * OR
  * LIKE (with % or _ wildcard)
  * BETWEEN
  * LIMIT

* Sorting and Grouping
  * ORDER BY
  * DISTINCT
  * GROUP BY

* Aggregates
  * MAX
  * MIN
  * SUM
  * AVG
  * COUNT



## Make some plots

Make some cool plots to go with your data. Write SQL queries to get ONLY the information you need for each plot. (Don't pull ALL the data and then just plot a few columns.)



## EXTRA CREDIT:
* Use a query that joins two tables.
* Make a model to see if you can predict something
* Come up with something else cool to do with your data
